In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def remove_race(race1,race2): #race_set(r,Q)=race_set(r,Q)-race_set(r,V)
    for key1 in race1.keys():
        try:
            dup=list(set(race1[key1] + race2[key1]))
            print(dup)
            for index in range(len(dup)):
                race1[key1].remove(dup[index])
        except Exception as e:
            print(e)
            continue
    return race1


variants=''
V=[]
#variants=GenerateVariants(Q,V0)

race_set1={'r1':['s2'],'r2':[],'r3':['s4'],'r4':[]}
race_set2={'r1':['s2'],'r2':[],'r3':['s4'],'r4':[]}

#while !variants:
    

In [ ]:
def Generatevariants(Q,V):
    for key in V.keys():
        if key[0]='r': #最初の文字でreceiveか判定
            remove(race_set,)

In [12]:
race_set1={'r1':['s2'],'r2':[],'r3':['s4'],'r4':[]}
race_set2={'r1':['s2'],'r2':[],'r3':[''],'r4':[]}

race_set1.update(race_set2)

In [51]:
race_set1={'r1':['s2'],'r2':[],'r3':['s4'],'r4':[]}
race_set2={'r1':['s2'],'r2':[],'r3':['s4'],'r4':[]}

In [53]:
remove_race(race_set1,race_set2)

['s2']
[]
['s4']
[]


{'r1': [], 'r2': [], 'r3': [], 'r4': []}

In [37]:
race=list(set(race_set1['r1'] + race_set2['r1']))

In [41]:
race

['s2']